# Why Customers Leave: A Data‑Driven Churn Analysis

In this analysis we construct a synthetic dataset for a telecom company. Each row represents a customer with attributes like tenure, monthly charges and contract type. We build a logistic regression model to predict whether a customer will churn.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

np.random.seed(99)
n_customers = 1000

data = pd.DataFrame({
    "tenure_months": np.random.randint(1, 60, size=n_customers),
    "monthly_charges": np.random.uniform(20, 100, size=n_customers),
    "total_charges": lambda df: df["tenure_months"] * df["monthly_charges"] + np.random.normal(0, 100, size=n_customers),
    "contract_type": np.random.choice(["Month-to-month", "One year", "Two year"], size=n_customers, p=[0.6, 0.25, 0.15]),
    "payment_method": np.random.choice(["Electronic", "Mailed check", "Bank transfer"], size=n_customers),
    "internet_service": np.random.choice(["DSL", "Fiber optic", "None"], size=n_customers, p=[0.4, 0.4, 0.2])
})

# Generate churn as a function of tenure and contract type
prob_churn = 0.4 - 0.005 * data["tenure_months"]
prob_churn += data["contract_type"].map({"Month-to-month": 0.1, "One year": -0.05, "Two year": -0.1})
data["churn"] = np.random.binomial(1, p=prob_churn.clip(0.05, 0.8))

data.head()


We preprocess the dataset by one‑hot encoding categorical features and then train a logistic regression model. We evaluate the model on a held‑out test set and display the confusion matrix.

In [ ]:

# One-hot encode categorical variables
data_dummies = pd.get_dummies(data.drop(columns=["churn"]), drop_first=True)
X = data_dummies
y = data["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks([0,1], ["Not churn", "Churn"])
plt.yticks([0,1], ["Not churn", "Churn"])
plt.ylabel("True label")
plt.xlabel("Predicted label")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i,j], ha='center', va='center')

plt.tight_layout()
plt.savefig("churn_confusion_matrix.png")


This model provides a baseline for predicting churn. The confusion matrix above shows how well the model distinguishes between customers who stay and those who leave.